# Cash Flow Forecasting for Treasury

## Use Case Overview

**Business Problem:** Treasury teams need accurate cash flow forecasts to:
- Ensure sufficient liquidity for operations
- Optimize investment of excess cash
- Plan borrowing requirements
- Manage FX exposures

**Traditional Approach:** Excel-based forecasts using historical averages → 50-60% accuracy

**AI Approach:** ML models capturing seasonality, trends, and patterns → 85-95% accuracy

---

## Models We'll Explore

| Model | Type | Strengths | Complexity |
|-------|------|-----------|------------|
| **Prophet** | Additive | Handles seasonality, holidays | Low |
| **ARIMA** | Statistical | Well-understood, interpretable | Medium |
| **LSTM** | Deep Learning | Captures complex patterns | High |

---

## Learning Objectives

By the end of this notebook, you will:
1. Generate realistic treasury cash flow data
2. Perform exploratory data analysis on cash positions
3. Build and compare forecasting models
4. Evaluate model performance
5. Visualize forecasts with uncertainty bands

## 1. Setup and Imports

In [ ]:
# Install required packages (uncomment if needed)
# !pip install pandas numpy matplotlib seaborn prophet scikit-learn plotly

In [ ]:
# Core imports
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ML/Forecasting
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print("✅ All imports successful!")

In [ ]:
# Add parent directory to path for imports
import sys
sys.path.append('..')

# Import our custom generators
from src.treasury_sim.generators import (
    generate_cash_flows,
    generate_daily_cash_position,
    set_seed
)

print("✅ Custom generators loaded!")

## 2. Generate Synthetic Cash Flow Data

### Thinking Trace 🧠

> **Why synthetic data?**
> - Real treasury data is highly confidential
> - Synthetic data allows us to demonstrate concepts
> - We can control patterns to validate our models
>
> **What patterns should we include?**
> - **Weekly seasonality**: Less activity on weekends
> - **Monthly seasonality**: Spikes at month-end (payments, collections)
> - **Quarterly patterns**: Tax payments, dividend flows
> - **Trend**: Gradual business growth
> - **Noise**: Random daily variations

In [ ]:
# Generate 2 years of cash flow data
set_seed(42)

# Transaction-level data
cash_flows = generate_cash_flows(days=730, n_accounts=5, seed=42)

# Daily position data
daily_positions = generate_daily_cash_position(days=730, n_accounts=5, seed=42)

print(f"📊 Generated {len(cash_flows):,} transactions")
print(f"📊 Generated {len(daily_positions):,} daily position records")
print(f"\n📅 Date range: {cash_flows['date'].min().date()} to {cash_flows['date'].max().date()}")

In [ ]:
# Preview the data
print("=" * 60)
print("CASH FLOW TRANSACTIONS (Sample)")
print("=" * 60)
cash_flows.head(10)

In [ ]:
# Summary statistics
print("\n📈 SUMMARY STATISTICS")
print("=" * 60)
print(f"\nBy Transaction Type:")
print(cash_flows.groupby('type')['amount'].agg(['count', 'sum', 'mean']).round(2))

print(f"\nBy Category:")
print(cash_flows.groupby('category')['amount'].agg(['count', 'sum']).round(2))

## 3. Exploratory Data Analysis

### Thinking Trace 🧠

> **Key questions to answer:**
> 1. What is the overall cash flow trend?
> 2. Are there clear seasonal patterns?
> 3. What is the volatility of daily cash flows?
> 4. Are there outliers we need to handle?
>
> **Visualization strategy:**
> - Time series plots for trend
> - Box plots for distribution by day/month
> - Heatmaps for patterns

In [ ]:
# Aggregate to daily net cash flow
daily_net = cash_flows.groupby('date')['amount'].sum().reset_index()
daily_net.columns = ['date', 'net_cash_flow']
daily_net['cumulative'] = daily_net['net_cash_flow'].cumsum()

# Add time features
daily_net['day_of_week'] = daily_net['date'].dt.dayofweek
daily_net['day_of_month'] = daily_net['date'].dt.day
daily_net['month'] = daily_net['date'].dt.month
daily_net['week'] = daily_net['date'].dt.isocalendar().week

print(f"Daily net cash flow data: {len(daily_net)} records")
daily_net.head()

In [ ]:
# Create comprehensive visualization
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        'Daily Net Cash Flow', 'Cumulative Cash Position',
        'Distribution by Day of Week', 'Distribution by Day of Month',
        'Monthly Aggregates', '30-Day Rolling Average'
    ),
    vertical_spacing=0.12
)

# 1. Daily net cash flow
fig.add_trace(
    go.Scatter(x=daily_net['date'], y=daily_net['net_cash_flow'],
               mode='lines', name='Daily Net', line=dict(width=0.5)),
    row=1, col=1
)

# 2. Cumulative position
fig.add_trace(
    go.Scatter(x=daily_net['date'], y=daily_net['cumulative'],
               mode='lines', name='Cumulative', fill='tozeroy'),
    row=1, col=2
)

# 3. Box plot by day of week
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
for i, day in enumerate(days):
    day_data = daily_net[daily_net['day_of_week'] == i]['net_cash_flow']
    fig.add_trace(
        go.Box(y=day_data, name=day, showlegend=False),
        row=2, col=1
    )

# 4. Average by day of month
dom_avg = daily_net.groupby('day_of_month')['net_cash_flow'].mean()
fig.add_trace(
    go.Bar(x=dom_avg.index, y=dom_avg.values, name='Avg by DOM', showlegend=False),
    row=2, col=2
)

# 5. Monthly aggregates
monthly = daily_net.groupby(daily_net['date'].dt.to_period('M'))['net_cash_flow'].sum()
fig.add_trace(
    go.Bar(x=[str(p) for p in monthly.index], y=monthly.values, name='Monthly Total', showlegend=False),
    row=3, col=1
)

# 6. Rolling average
daily_net['rolling_30'] = daily_net['net_cash_flow'].rolling(30).mean()
fig.add_trace(
    go.Scatter(x=daily_net['date'], y=daily_net['rolling_30'],
               mode='lines', name='30-Day MA', line=dict(width=2)),
    row=3, col=2
)

fig.update_layout(height=900, title_text='Cash Flow Exploratory Analysis', showlegend=False)
fig.show()

### 📊 Key Insights from EDA

1. **Weekend Effect**: Clear reduction in cash flow activity on weekends
2. **Month-End Spike**: Day 25-31 shows higher outflows (salary, payments)
3. **Overall Trend**: Slight negative trend due to more outflows than inflows
4. **Volatility**: High daily volatility, but smooths over 30-day window

**Implication for Modeling**: We need models that can capture multiple seasonality patterns (weekly + monthly)

## 4. Data Preparation for Forecasting

### Thinking Trace 🧠

> **Train/Test Split Strategy:**
> - Use last 90 days as test set (3 months forecast horizon)
> - Train on remaining ~21 months
> - This simulates real-world quarterly forecasting
>
> **Feature Engineering:**
> - Lag features (yesterday, last week, last month)
> - Rolling statistics (7-day, 30-day averages)
> - Calendar features (day of week, month, holidays)

In [ ]:
# Prepare data for Prophet (requires 'ds' and 'y' columns)
prophet_df = daily_net[['date', 'net_cash_flow']].copy()
prophet_df.columns = ['ds', 'y']

# Train/test split (last 90 days for testing)
train_size = len(prophet_df) - 90
train_df = prophet_df.iloc[:train_size]
test_df = prophet_df.iloc[train_size:]

print(f"📚 Training set: {len(train_df)} days ({train_df['ds'].min().date()} to {train_df['ds'].max().date()})")
print(f"🧪 Test set: {len(test_df)} days ({test_df['ds'].min().date()} to {test_df['ds'].max().date()})")

## 5. Model 1: Prophet

### Why Prophet?

Prophet is Facebook's open-source forecasting tool, ideal for:
- **Multiple seasonality** (daily, weekly, yearly)
- **Holiday effects** (we can add payment dates, quarter-ends)
- **Trend changes** (handles changepoints)
- **Missing data** (robust to gaps)

### Model Formula

$$y(t) = g(t) + s(t) + h(t) + \epsilon_t$$

Where:
- $g(t)$ = trend (linear or logistic growth)
- $s(t)$ = seasonality (Fourier series)
- $h(t)$ = holiday effects
- $\epsilon_t$ = error term

In [ ]:
try:
    from prophet import Prophet
    PROPHET_AVAILABLE = True
    print("✅ Prophet is available")
except ImportError:
    PROPHET_AVAILABLE = False
    print("⚠️ Prophet not installed. Install with: pip install prophet")
    print("   Skipping Prophet model...")

In [ ]:
if PROPHET_AVAILABLE:
    # Initialize Prophet with custom seasonality
    model_prophet = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,  # Too granular for daily data
        seasonality_mode='multiplicative',  # Better for financial data
        changepoint_prior_scale=0.05,  # Flexibility of trend
        interval_width=0.95  # 95% confidence interval
    )
    
    # Add monthly seasonality
    model_prophet.add_seasonality(
        name='monthly',
        period=30.5,
        fourier_order=5
    )
    
    # Fit the model
    print("🔧 Training Prophet model...")
    model_prophet.fit(train_df)
    print("✅ Prophet model trained!")

In [ ]:
if PROPHET_AVAILABLE:
    # Make predictions
    future = model_prophet.make_future_dataframe(periods=90)
    forecast_prophet = model_prophet.predict(future)
    
    # Extract test period predictions
    prophet_pred = forecast_prophet.iloc[-90:][['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    prophet_pred = prophet_pred.merge(test_df, on='ds')
    
    # Calculate metrics
    mae_prophet = mean_absolute_error(prophet_pred['y'], prophet_pred['yhat'])
    rmse_prophet = np.sqrt(mean_squared_error(prophet_pred['y'], prophet_pred['yhat']))
    mape_prophet = mean_absolute_percentage_error(prophet_pred['y'].abs(), prophet_pred['yhat'].abs()) * 100
    
    print("📊 PROPHET MODEL PERFORMANCE")
    print("=" * 40)
    print(f"MAE:  ${mae_prophet:,.0f}")
    print(f"RMSE: ${rmse_prophet:,.0f}")
    print(f"MAPE: {mape_prophet:.1f}%")

In [ ]:
if PROPHET_AVAILABLE:
    # Visualize Prophet results
    fig = go.Figure()
    
    # Training data
    fig.add_trace(go.Scatter(
        x=train_df['ds'], y=train_df['y'],
        mode='lines', name='Training Data',
        line=dict(color='blue', width=1)
    ))
    
    # Test data (actuals)
    fig.add_trace(go.Scatter(
        x=test_df['ds'], y=test_df['y'],
        mode='lines', name='Actual (Test)',
        line=dict(color='green', width=2)
    ))
    
    # Predictions
    fig.add_trace(go.Scatter(
        x=prophet_pred['ds'], y=prophet_pred['yhat'],
        mode='lines', name='Prophet Forecast',
        line=dict(color='red', width=2, dash='dash')
    ))
    
    # Confidence interval
    fig.add_trace(go.Scatter(
        x=pd.concat([prophet_pred['ds'], prophet_pred['ds'][::-1]]),
        y=pd.concat([prophet_pred['yhat_upper'], prophet_pred['yhat_lower'][::-1]]),
        fill='toself', fillcolor='rgba(255,0,0,0.1)',
        line=dict(color='rgba(255,0,0,0)'),
        name='95% CI'
    ))
    
    fig.update_layout(
        title='Prophet Cash Flow Forecast vs Actuals',
        xaxis_title='Date',
        yaxis_title='Net Cash Flow ($)',
        height=500
    )
    fig.show()

## 6. Model 2: Simple Moving Average Baseline

### Why a Baseline?

Before using complex models, we should establish a baseline using simple methods.
This helps us understand if ML adds value.

**Baseline approaches:**
- **Naive**: Tomorrow = Today (random walk)
- **7-Day Average**: Tomorrow = Average of last 7 days
- **30-Day Average**: Tomorrow = Average of last 30 days

In [ ]:
# Prepare data for baseline models
baseline_df = daily_net[['date', 'net_cash_flow']].copy()
baseline_df['naive'] = baseline_df['net_cash_flow'].shift(1)
baseline_df['ma_7'] = baseline_df['net_cash_flow'].rolling(7).mean().shift(1)
baseline_df['ma_30'] = baseline_df['net_cash_flow'].rolling(30).mean().shift(1)

# Test set only
baseline_test = baseline_df.iloc[-90:].dropna()

# Calculate metrics for each baseline
baselines = {}
for model_name in ['naive', 'ma_7', 'ma_30']:
    mae = mean_absolute_error(baseline_test['net_cash_flow'], baseline_test[model_name])
    rmse = np.sqrt(mean_squared_error(baseline_test['net_cash_flow'], baseline_test[model_name]))
    baselines[model_name] = {'MAE': mae, 'RMSE': rmse}

print("📊 BASELINE MODEL PERFORMANCE")
print("=" * 40)
for name, metrics in baselines.items():
    print(f"{name:8s}: MAE = ${metrics['MAE']:,.0f}, RMSE = ${metrics['RMSE']:,.0f}")

## 7. Model Comparison Dashboard

### Thinking Trace 🧠

> **What makes a good forecast?**
> 1. **Accuracy** - Low MAE/RMSE/MAPE
> 2. **Reliability** - Consistent across different periods
> 3. **Calibration** - Confidence intervals contain actuals ~95% of time
> 4. **Interpretability** - Can we explain the forecast?

In [ ]:
# Compile all results
results = {
    'Naive (Yesterday)': baselines['naive'],
    '7-Day Moving Avg': baselines['ma_7'],
    '30-Day Moving Avg': baselines['ma_30'],
}

if PROPHET_AVAILABLE:
    results['Prophet'] = {'MAE': mae_prophet, 'RMSE': rmse_prophet}

# Create comparison table
results_df = pd.DataFrame(results).T
results_df['Improvement vs Naive'] = (
    (results_df.loc['Naive (Yesterday)', 'MAE'] - results_df['MAE']) / 
    results_df.loc['Naive (Yesterday)', 'MAE'] * 100
).round(1)

print("📊 MODEL COMPARISON")
print("=" * 60)
print(results_df.to_string())

In [ ]:
# Visualization of model comparison
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('MAE Comparison', 'RMSE Comparison')
)

models = list(results.keys())
mae_values = [results[m]['MAE'] for m in models]
rmse_values = [results[m]['RMSE'] for m in models]

# Color coding: Prophet in different color if best
colors = ['#636EFA'] * len(models)
if PROPHET_AVAILABLE and 'Prophet' in models:
    prophet_idx = models.index('Prophet')
    if mae_values[prophet_idx] == min(mae_values):
        colors[prophet_idx] = '#00CC96'

fig.add_trace(
    go.Bar(x=models, y=mae_values, marker_color=colors, name='MAE'),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=models, y=rmse_values, marker_color=colors, name='RMSE'),
    row=1, col=2
)

fig.update_layout(height=400, title_text='Forecasting Model Comparison', showlegend=False)
fig.show()

## 8. Business Application: Liquidity Planning

### Thinking Trace 🧠

> **How would treasury use this forecast?**
> 1. **Liquidity Buffer**: Use lower bound of CI to ensure minimum cash
> 2. **Investment Decisions**: Upper bound suggests potential excess cash
> 3. **Borrowing Planning**: If forecast shows deficits, arrange facilities
> 4. **Variance Analysis**: Compare forecast vs actual to improve model

In [ ]:
if PROPHET_AVAILABLE:
    # Generate 30-day forward forecast
    future_30 = model_prophet.make_future_dataframe(periods=30, include_history=False)
    future_30 = future_30[future_30['ds'] > prophet_df['ds'].max()]
    
    if len(future_30) > 0:
        forecast_30 = model_prophet.predict(future_30)
        
        # Calculate cumulative forecast
        forecast_30['cumulative_yhat'] = forecast_30['yhat'].cumsum()
        forecast_30['cumulative_lower'] = forecast_30['yhat_lower'].cumsum()
        forecast_30['cumulative_upper'] = forecast_30['yhat_upper'].cumsum()
        
        print("📈 30-DAY CASH FLOW FORECAST")
        print("=" * 60)
        print(f"\nExpected Net Cash Flow: ${forecast_30['yhat'].sum():,.0f}")
        print(f"Worst Case (5%):        ${forecast_30['yhat_lower'].sum():,.0f}")
        print(f"Best Case (95%):        ${forecast_30['yhat_upper'].sum():,.0f}")
        
        # Liquidity recommendation
        min_cash = forecast_30['cumulative_lower'].min()
        print(f"\n💰 LIQUIDITY RECOMMENDATION")
        print(f"Maintain minimum buffer of: ${abs(min_cash):,.0f}")

## 9. Key Takeaways

### What We Learned

1. **AI/ML can significantly improve cash flow forecasting accuracy** compared to simple averages
2. **Prophet is well-suited for treasury forecasting** due to its handling of multiple seasonalities
3. **Uncertainty quantification is critical** - point forecasts aren't enough for treasury decisions
4. **Feature engineering matters** - incorporating calendar effects improves performance

### Production Considerations

| Aspect | Recommendation |
|--------|----------------|
| **Data** | Integrate with SAP/ERP for real-time data |
| **Frequency** | Re-train weekly, predict daily |
| **Monitoring** | Track forecast accuracy, alert on drift |
| **Explainability** | Use Prophet's component plots for stakeholders |

### Next Steps

1. Add external features (FX rates, commodity prices, economic indicators)
2. Implement LSTM for capturing complex patterns
3. Build ensemble model combining Prophet + LSTM
4. Deploy as API for TMS integration

---

## Appendix: Model Selection Guide

| Scenario | Recommended Model | Why |
|----------|------------------|-----|
| Quick POC | Prophet | Easy setup, good defaults |
| High accuracy needed | LSTM Ensemble | Captures complex patterns |
| Interpretability critical | ARIMA | Well-understood statistics |
| Limited data (<1 year) | Prophet | Handles short series well |
| Real-time updates | Online ARIMA | Incremental learning |

---

*Notebook created for Treasury AI educational purposes*

*Author: Ozgur Guler (ozgur.guler1@gmail.com)*